### Targeted

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchattacks import PGD
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define transformations for CIFAR-100
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

# Initialize ResNet-18 model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 100)  # Adjust for CIFAR-100
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Step decay scheduler
def lr_schedule(epoch):
    if epoch < 80:
        return 1
    else:
        return 0.1
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_schedule)

# Define targeted PGD attack for training
attack = PGD(model, eps=8/255, alpha=2/255, steps=7, random_start=True, targeted=True)

# Adversarial training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Generate targeted adversarial examples
        target_labels = (labels + 1) % 100  # Simple target: next class
        adv_images = attack(images, target_labels)
        
        # Compute loss on clean and adversarial examples
        optimizer.zero_grad()
        clean_outputs = model(images)
        clean_loss = criterion(clean_outputs, labels)
        
        adv_outputs = model(adv_images)
        adv_loss = criterion(adv_outputs, labels)  # Train to predict true labels
        
        # Combined loss (50% clean, 50% adversarial)
        loss = 0.5 * clean_loss + 0.5 * adv_loss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")
    
    # Update learning rate
    scheduler.step()
    print(f"Epoch {epoch+1}, Average Loss: {running_loss / len(train_loader):.4f}")

# Save the adversarially trained model
torch.save(model.state_dict(), 'resnet18_cifar100_targeted_adv.pth')

# Basic evaluation (extend with your evaluate_full_test for detailed metrics)
model.eval()
clean_correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        clean_correct += predicted.eq(labels).sum().item()
clean_acc = 100. * clean_correct / total
print(f"Adversarial Trained - Clean Accuracy: {clean_acc:.2f}%")

### Untargeted

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchattacks import PGD
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define transformations for CIFAR-100
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load CIFAR-100 dataset
train_dataset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

# Initialize ResNet-18 model
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 100)  # Adjust for CIFAR-100
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

# Step decay scheduler
def lr_schedule(epoch):
    if epoch < 80:
        return 1
    else:
        return 0.1
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_schedule)

# Define targeted PGD attack for training
attack = PGD(model, eps=8/255, alpha=2/255, steps=7, random_start=True, targeted=True)

# Adversarial training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Generate targeted adversarial examples
        target_labels = (labels + 1) % 100  # Simple target: next class
        adv_images = attack(images, target_labels)
        
        # Compute loss on clean and adversarial examples
        optimizer.zero_grad()
        clean_outputs = model(images)
        clean_loss = criterion(clean_outputs, labels)
        
        adv_outputs = model(adv_images)
        adv_loss = criterion(adv_outputs, labels)  # Train to predict true labels
        
        # Combined loss (50% clean, 50% adversarial)
        loss = 0.5 * clean_loss + 0.5 * adv_loss
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss.item():.4f}")
    
    # Update learning rate
    scheduler.step()
    print(f"Epoch {epoch+1}, Average Loss: {running_loss / len(train_loader):.4f}")

# Save the adversarially trained model
torch.save(model.state_dict(), 'resnet18_cifar100_targeted_adv.pth')

# Basic evaluation
model.eval()
clean_correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        clean_correct += predicted.eq(labels).sum().item()
clean_acc = 100. * clean_correct / total
print(f"Adversarial Trained - Clean Accuracy: {clean_acc:.2f}%")